In [1]:
import pandas
import numpy as np

In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest,f_classif,SelectFdr
from sklearn import svm
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model, Sequential
from keras import backend
from keras import regularizers
from keras.layers import Dropout
import keras
from matplotlib import pyplot as plt
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import normalize
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.decomposition import PCA

Using TensorFlow backend.


In [3]:
# read data
methy = pandas.read_csv("autoencoder data/methylation.csv")
mrna = pandas.read_csv("autoencoder data/mrna.csv")
mirna = pandas.read_csv("autoencoder data/mirna.csv")
clinical_new = pandas.read_csv("clinical_data.csv")

# 
clinical_new = clinical_new.drop(['Unnamed: 0'], axis = 1)
clinical_new = clinical_new[['bcr_patient_barcode','vital_status','survival']]
#clinical_new = clinical_new.set_index(['bcr_patient_barcode'])

methy = methy.drop(['Unnamed: 0'], axis=1)
mrna = mrna.drop(['Unnamed: 0'], axis=1)
mirna = mirna.drop(['Unnamed: 0'], axis=1)

methy = methy.set_index(['Group.1'])
mrna = mrna.set_index(['Group.1'])
mirna = mirna.set_index(['GeneSymbol'])

methy = methy.transpose()
mrna = mrna.transpose()
mirna = mirna.transpose()

# data log2 transformation
mrna = np.log2(mrna+1)
mirna = np.log2(mirna+1)

In [4]:
clinical_new = pandas.read_csv("clinical_data3.csv")
clinical_new = clinical_new.drop(['Unnamed: 0'], axis = 1)
clinical_new = clinical_new[['bcr_patient_barcode','vital_status','survival']]
data_all = pandas.concat([methy,mrna,mirna],axis = 1)
data_all2 = data_all.loc[clinical_new['bcr_patient_barcode'],:]
methy = data_all2.iloc[:,0:20980]
mrna = data_all2.iloc[:,20980:38168]
mirna = data_all2.iloc[:,38168:38597]

In [5]:
methy_train = normalize(methy, norm='l2',axis = 1)

mrna_train = normalize(mrna, norm='l2',axis = 1)

mirna_train = normalize(mirna, norm='l2',axis = 1)

In [6]:
data = pandas.concat([pandas.DataFrame(methy_train),pandas.DataFrame(mrna_train),pandas.DataFrame(mirna_train)],axis = 1)

In [7]:
data

,0,1,2,3,4,5,6,7,8,9,...,419,420,421,422,423,424,425,426,427,428
0,0.006003,0.006003,0.004392,0.007359,0.007479,0.008025,0.004393,0.003431,0.008051,0.007759,...,0.013157,0.002459,0.013994,0.007694,0.004482,0.022254,0.038162,0.044174,0.070889,0.118764
1,0.006637,0.006637,0.006748,0.007456,0.007355,0.007779,0.005634,0.002140,0.007796,0.007792,...,0.006868,0.009225,0.012340,0.012875,0.004989,0.020658,0.025679,0.044091,0.073041,0.108770
2,0.006122,0.006122,0.006193,0.007180,0.007408,0.007684,0.004890,0.003624,0.007848,0.007852,...,0.002360,0.002360,0.007515,0.012110,0.010030,0.019145,0.026498,0.044723,0.078058,0.118641
3,0.006762,0.006762,0.006562,0.007235,0.007409,0.007835,0.004785,0.001251,0.007933,0.007887,...,0.000000,0.002201,0.004072,0.014971,0.005699,0.005699,0.014971,0.039724,0.089113,0.117408
4,0.007053,0.007053,0.006131,0.007297,0.007169,0.007777,0.004685,0.003617,0.007831,0.007787,...,0.009661,0.005101,0.003629,0.012960,0.006413,0.027249,0.028892,0.043821,0.075957,0.115636
5,0.006854,0.006854,0.006631,0.007344,0.007405,0.007785,0.004295,0.001230,0.007847,0.007818,...,0.011963,0.000000,0.009956,0.018671,0.009956,0.020526,0.030209,0.047062,0.072592,0.111792
6,0.006841,0.006841,0.006393,0.007184,0.007459,0.007769,0.005401,0.001372,0.007819,0.007773,...,0.004798,0.006031,0.009941,0.014052,0.008154,0.028007,0.027495,0.045788,0.077521,0.109528
7,0.006315,0.006315,0.006416,0.007486,0.007444,0.007841,0.005124,0.002140,0.007837,0.007841,...,0.003842,0.005847,0.005847,0.034306,0.009057,0.019812,0.024425,0.042071,0.084338,0.111378
8,0.006631,0.006631,0.006271,0.007221,0.007256,0.007703,0.004780,0.003695,0.007805,0.007795,...,0.002357,0.000000,0.007447,0.020214,0.016492,0.015847,0.024985,0.042530,0.076787,0.115560
9,0.006481,0.006481,0.006595,0.007333,0.007455,0.007790,0.005104,0.002183,0.007800,0.007770,...,0.007636,0.002387,0.008331,0.014171,0.001250,0.016837,0.011325,0.042478,0.087019,0.124264


In [8]:
pca = PCA(n_components=100)
principalComponents = pca.fit_transform(data)
principalDf = pandas.DataFrame(principalComponents)
principalDf

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.052888,-0.030838,-0.086307,0.053705,-0.108754,0.041231,0.036629,0.023586,-0.014135,0.023393,...,0.002108,-0.000693,0.004251,-0.001453,0.000367,-0.002348,0.007594,-0.002044,-0.004694,0.000678
1,0.064976,-0.055861,0.006576,-0.013093,0.047871,-0.027940,0.014670,0.033821,-0.013558,0.034050,...,0.011854,0.006479,0.001573,-0.009491,0.005929,-0.006843,-0.007455,0.004950,0.004586,0.002640
2,-0.061733,0.060477,-0.026753,-0.012163,0.003095,-0.031094,0.016781,-0.003328,-0.028973,0.035229,...,0.007520,-0.002697,-0.004067,0.016809,-0.000817,0.000510,-0.011387,-0.010689,0.010072,-0.001020
3,0.027460,-0.055310,0.009629,-0.022639,0.006797,-0.063538,0.016166,-0.042444,-0.012594,0.083486,...,0.014181,-0.007729,-0.008361,-0.000193,0.005947,0.002264,-0.003466,0.001601,-0.002637,-0.004675
4,0.050181,-0.013531,0.023991,-0.010720,-0.014698,-0.046088,0.007438,0.012180,-0.027415,0.022976,...,-0.002904,-0.001018,-0.000526,0.007125,-0.004751,-0.001375,-0.007506,0.003975,0.005895,0.006517
5,-0.023376,0.003139,0.018069,0.038391,-0.006730,0.031679,-0.037507,0.021119,-0.001769,0.036472,...,-0.006302,0.000752,0.007203,-0.003557,-0.017891,0.011439,-0.002714,-0.006499,0.004882,0.000076
6,-0.064836,0.061168,0.042763,0.062256,-0.057769,0.018906,-0.002554,0.002284,-0.001872,0.049015,...,-0.008813,-0.007550,-0.001137,0.001229,0.003934,0.005759,0.003275,-0.008391,0.003208,0.015907
7,0.010947,-0.024344,0.020178,0.007888,-0.048746,-0.021148,0.002046,-0.004042,-0.021466,0.025677,...,0.000330,-0.004069,-0.001535,0.005152,0.015307,-0.004127,0.004882,-0.006334,0.006274,-0.003989
8,-0.079356,0.021879,-0.029965,0.019451,0.006085,0.002762,-0.012127,0.009463,0.002985,0.057247,...,0.004545,0.003334,0.007785,0.002409,0.001028,0.006891,0.004410,-0.004393,0.004530,-0.005384
9,0.001746,-0.075903,0.038009,-0.043726,0.074309,-0.036674,0.043034,-0.034702,0.036480,0.013545,...,0.008764,0.002754,0.010374,0.002052,0.003280,-0.003126,0.001909,-0.001740,-0.007858,0.013109


In [9]:
def coxph_feature_selection(data,clinical_data):
    p_value = []
    #adj_p_value = []
    cph = CoxPHFitter()
    for j in range(0,100):
        data_label = pandas.concat([data[j],clinical_data[["survival","vital_status"]]],axis = 1)
        cph.fit(data_label, duration_col="survival", event_col="vital_status")
        # get p value
        #p_value.append(cph.summary.iloc[0,4])
        if cph.summary.iloc[0,4] <0.05:
            p_value.append(j)
    data_new = data[p_value]
    print(len(p_value))    
    # adjust p value
    #adj_p = fdrcorrection(p_value)[1]
    #print(p_value,adj_p)
    #for j in range(0,100):
        #if adj_p[j] <0.05:
            #adj_p_value.append(j)
    #data_new = data[adj_p_value]
    return data_new   

In [10]:
def kmeans_function(data,cluster):
    for n in range(2,10):
        kmeans = KMeans(n_clusters=n, random_state=10).fit(data)
        labels = kmeans.labels_
        print(silhouette_score(data, labels))
        print(calinski_harabaz_score(data, labels))
    kmeans = KMeans(n_clusters=cluster, n_init=10).fit(data)
    labels = kmeans.labels_
    return labels

In [11]:
from lifelines import CoxPHFitter

In [12]:
from sklearn.cluster import KMeans

In [13]:
from sklearn.metrics import confusion_matrix,accuracy_score,silhouette_score,calinski_harabaz_score

In [14]:
df_new = coxph_feature_selection(principalDf,clinical_new) 

C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [62] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [63] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [64] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [65] have very low variance. This may harm

C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [92] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [93] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [94] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [95] have very low variance. This may harm

4


C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [98] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:885: ConvergenceWarning: Column(s) [99] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.
  warnings.warn(warning_text, ConvergenceWarning)


In [17]:
df_new.to_csv("output_cv/df_pca.csv")

In [18]:
labels = kmeans_function(df_new,3)
labels

0.3263738513142622
80.90873426798721
0.33339527561222726
86.952256555055
0.3287214326777778
85.3400985797705
0.29835519852962616
77.89415420149525
0.32348219774323833
77.31008239565625
0.3123245939147273
75.58235795512239
0.29821558010926313
80.52983450154032
0.29806100638774863
83.2178162439326


array([1, 2, 2, 2, 2, 0, 0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 0, 0, 2,
       0, 2, 1, 2, 1, 1, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0, 0, 0, 0, 1, 2, 0,
       2, 2, 1, 2, 0, 1, 1, 1, 0, 0, 2, 1, 2, 0, 1, 2, 0, 0, 0, 1, 0, 2,
       0, 0, 2, 2, 2, 0, 2, 1, 0, 1, 0, 1, 1, 2, 0, 0, 1, 2, 1, 1, 1, 1,
       2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 1, 1, 2, 2, 1, 2, 2, 2, 0,
       2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 1, 2, 0, 0])

In [17]:
principalDf.to_csv("pca.csv")

svm

In [17]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest,f_classif,SelectFdr
from sklearn import svm
from matplotlib import pyplot as plt
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix

In [18]:
methy_train = RobustScaler().fit_transform(methy_train)
methy_test = RobustScaler().fit_transform(methy_test)

mrna_train = RobustScaler().fit_transform(mrna_train)
mrna_test = RobustScaler().fit_transform(mrna_test)

mirna_train = RobustScaler().fit_transform(mirna_train)
mirna_test = RobustScaler().fit_transform(mirna_test)

NameError: name 'methy_train1' is not defined

In [28]:
methy_train = normalize(methy_train, norm='l2',axis = 0)
methy_test = normalize(methy_test, norm='l2',axis = 0)

mrna_train = normalize(mrna_train, norm='l2',axis = 0)
mrna_test = normalize(mrna_test, norm='l2',axis = 0)

mirna_train = normalize(mirna_train, norm='l2',axis = 0)
mirna_test = normalize(mirna_test, norm='l2',axis = 0)

In [37]:
data_new = [0] * 11
for k in range(5,60,5):
    methy_f = SelectKBest(f_classif,k).fit_transform(methy_train,labels) 
    
    mrna_f = SelectKBest(f_classif,k*2).fit_transform(mrna_train,labels)

    mirna_f = SelectKBest(f_classif,k).fit_transform(mirna_train,labels)
    
    i = int(k/5-1)
    data_new[i] = pandas.concat([pandas.DataFrame(methy_f),pandas.DataFrame(mrna_f),pandas.DataFrame(mirna_f)],axis = 1)

In [38]:
result = [0] * 11

for i in range(0,11):
    svm_parameters = {
    #'kernel': ['linear','poly','rbf','sigmoid'],
    'gamma': [0.01,0.05,0.1,0.5],
    'coef0': [0.01,0.1,1],
    'C': [0.1,0.5,1],
    #'degree' : [2,3] 
    }
    svm_tune = GridSearchCV(estimator=svm.SVC('sigmoid'),param_grid=svm_parameters,cv=3)
    svm_tune.fit(data_new[i],labels)
    result[i] = cross_val_score(svm_tune, data_new[i],labels, cv=3)

C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: Deprecation

C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: Deprecation

In [39]:
for i in range(0,11):
    print(np.mean(result[i]))

0.9758033294618661
0.9758033294618661
0.983739837398374
0.983739837398374
0.983739837398374
0.983739837398374
0.9758033294618661
0.9758033294618661
0.9758033294618661
0.9758033294618661
0.9758033294618661


In [50]:
methy_train = pandas.DataFrame(methy_train)

methy_f = SelectKBest(f_classif, k=15).fit(methy_train,labels)
methy_f = methy_f.get_support()
methy_new = methy_train.loc[:,methy_f]

In [51]:
mrna_train = pandas.DataFrame(mrna_train)

mrna_f = SelectKBest(f_classif, k=30).fit(mrna_train,labels).get_support()
mrna_new = mrna_train.loc[:,mrna_f]
mrna_new.shape

(125, 30)

In [52]:
mirna_train = pandas.DataFrame(mirna_train)

mirna_f = SelectKBest(f_classif, k=15).fit(mirna_train,labels).get_support()
mirna_new = mirna_train.loc[:,mirna_f]
mirna_new.shape

(125, 15)

In [53]:
data_new1 = pandas.concat([methy_new,mrna_new,mirna_new],axis = 1)

In [54]:
svm_parameters = {
'kernel': ['linear','poly','rbf','sigmoid'],
'gamma': [0.05,0.1,1],
'coef0': [0.01,0.1,0.2,0.001],
'C': [0.1,0.05,0.5,1,1.2],
'degree' : [2,3,4] 
}
svm_tune = GridSearchCV(estimator=svm.SVC(),param_grid=svm_parameters,cv=3)
svm_tune.fit(data_new1,labels)# data_new1 # limma_train
cross_val_score(svm_tune, data_new1,labels, cv=3)

C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\sklearn\model_selection\_search.py:841: Deprecation

array([1.        , 0.97619048, 0.95121951])

In [55]:
svm_tune.best_params_

{'C': 0.5, 'coef0': 0.2, 'degree': 4, 'gamma': 1, 'kernel': 'poly'}

In [56]:
methy_test = pandas.DataFrame(methy_test)
mrna_test = pandas.DataFrame(mrna_test)
mirna_test = pandas.DataFrame(mirna_test)

methy_t = methy_test[methy_new.columns]
mrna_t = mrna_test[mrna_new.columns]
mirna_t = mirna_test[mirna_new.columns]
test_new = pandas.concat([methy_t,mrna_t,mirna_t],axis = 1)
test_new.shape

(52, 60)

In [57]:
svm_tune_pre = svm_tune.predict(test_new)#test_new

In [58]:
confusion_matrix(svm_tune.predict(data_new1),labels)

array([[114,   1],
       [  0,  10]], dtype=int64)

In [59]:
svm_tune_pre

array([1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1], dtype=int64)